In [3]:
import spotipy
import json
import webbrowser
import urllib.request
import overpass
import spotipy.util as util

In [4]:
api = overpass.API(timeout=600)

response = api.get('node["name"="Providence"]', responseformat="json")
print(response["elements"][1])

{'type': 'node', 'id': 150980257, 'lat': 41.7062699, 'lon': -111.817638, 'tags': {'ele': '1401', 'gnis:feature_id': '1444657', 'name': 'Providence', 'place': 'village', 'population': '5540', 'population:date': '2006', 'source:population': 'US Census', 'wikidata': 'Q482412', 'wikipedia': 'en:Providence, Utah'}}


In [5]:
with open("ticketmaster_keys.txt", "r") as key_file:
    ticketmaster_key = key_file.read()

In [13]:
geohash = "gcpuv"
url = f"https://app.ticketmaster.com/discovery/v2/events.json?size=20&classificationName=music&geoPoint={geohash}&apikey={ticketmaster_key}"

tm_request = urllib.request.Request(url)
tm_response = urllib.request.urlopen(tm_request)

full_event_listings = json.loads(tm_response.read())
event_names = full_event_listings['_embedded']['events']
print(event_names)


[{'name': 'Paul McCartney', 'type': 'event', 'id': '17u8vOG6u3qwf3R', 'test': False, 'url': 'https://www.ticketmaster.co.uk/paul-mccartney-london-18-12-2024/event/350060D49DC944AF', 'locale': 'en-us', 'images': [{'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/5a0/ca8fbbf7-4803-4af6-8b18-50bbff2695a0_TABLET_LANDSCAPE_3_2.jpg', 'width': 1024, 'height': 683, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/5a0/ca8fbbf7-4803-4af6-8b18-50bbff2695a0_RETINA_PORTRAIT_16_9.jpg', 'width': 640, 'height': 360, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/5a0/ca8fbbf7-4803-4af6-8b18-50bbff2695a0_EVENT_DETAIL_PAGE_16_9.jpg', 'width': 205, 'height': 115, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/ac1/3bc6b045-e04f-4520-b1de-b552e2b1cac1_SOURCE', 'width': 2426, 'height': 1365, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/5a0/ca8fbbf7-4803-4af6-8b18-50bbff2695a0_RETINA_LANDSCAPE_1

In [28]:
artist_names = []

for artist in event_names:
    print(artist['name'])

Paul McCartney
Paul McCartney
Imagine Dragons: LOOM World Tour 2025
Imagine Dragons: LOOM World Tour 2025
Paul McCartney
Paul McCartney - Official VIP Ticket and Hotel Experiences
Paul McCartney
Paul McCartney - Official VIP Ticket and Hotel Experiences
Werchter Boutique 2025
Imagine Dragons: LOOM World Tour | VIP Packages
Imagine Dragons: LOOM World Tour
The Holiday: the Film with Live Orchestra
The Holiday: the Film with Live Orchestra
The Holiday - Film With Live Orchestra
The Holiday: the Film with Live Orchestra
Age UK's Love Christmas 2024
Wes Banderson
The Holiday: Film with Live Orchestra
The Holiday: the Film with Live Orchestra
STEP INTO CHRISTMAS
